In [3]:
import geopy
import folium
import requests
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Polygon
from IPython.display import display

# Función para obtener los límites de la colonia
def obtener_limites_colonia(nombre_colonia):
    geolocator = Nominatim(user_agent="mi-aplicacion", timeout=10)
    
    try:
        # Intentar obtener las coordenadas de la colonia
        location = geolocator.geocode(f"{nombre_colonia}, Monterrey, Nuevo León, México")
        
        if not location:
            return None, None
        
        # Usar la API de Overpass para obtener los límites de la colonia desde OpenStreetMap
        overpass_url = "http://overpass-api.de/api/interpreter"
        overpass_query = f"""
        [out:json];
        area["name"="{nombre_colonia}"]["admin_level"="10"];
        (way(area);
        rel(area););
        out body;
        >;
        out skel qt;
        """
        
        response = requests.get(overpass_url, params={'data': overpass_query}, timeout=10)
        data = response.json()
        
        # Revisar si se encontraron elementos en la respuesta
        ways = [element for element in data['elements'] if element['type'] == 'way' and 'nodes' in element]
        if not ways:
            return location, None
        
        # Obtener las coordenadas de los nodos
        nodes = {element['id']: (element['lat'], element['lon']) for element in data['elements'] if element['type'] == 'node'}
        
        # Construir los polígonos de la colonia
        polygons = []
        for way in ways:
            polygon_coords = [nodes[node_id] for node_id in way['nodes'] if node_id in nodes]
            if len(polygon_coords) > 2:  # Para ser un polígono válido, debe tener al menos 3 puntos
                polygons.append(Polygon(polygon_coords))
        
        if not polygons:
            return location, None
        
        # Crear un GeoDataFrame a partir de los polígonos y definir el CRS
        gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")  # Definir CRS WGS84
        
        return location, gdf
    
    except (requests.exceptions.Timeout, requests.exceptions.RequestException):
        print(f"Error al obtener los datos de la colonia '{nombre_colonia}'.")
        return None, None

# Función para mostrar el mapa interactivo
def mostrar_mapa_colonia(nombre_colonia):
    location, gdf = obtener_limites_colonia(nombre_colonia)
    
    if not location:
        print(f"No se encontró la colonia: {nombre_colonia}")
        return
    
    if gdf is None or gdf.empty:
        print(f"No se encontraron los límites de la colonia: {nombre_colonia}")
        return
    
    # Crear el mapa centrado en la colonia
    mapa = folium.Map(location=[location.latitude, location.longitude], zoom_start=15)
    
    # Convertir el GeoDataFrame a un formato GeoJSON
    geojson = gdf.to_json()
    
    # Función de estilo básica para la capa GeoJson
    def estilo_feature(feature):
        return {
            'fillColor': '#ff7800',  # Color de llenado
            'color': '#ff7800',  # Color del borde
            'weight': 3,  # Grosor del borde
            'fillOpacity': 0.5  # Opacidad del color de llenado
        }
    
    # Agregar los límites de la colonia al mapa con estilo
    folium.GeoJson(
        geojson,
        style_function=estilo_feature
    ).add_to(mapa)
    
    # Mostrar el mapa en el notebook
    display(mapa)

# Pedir al usuario el nombre de la colonia
nombre_colonia = 'San Francisco de Asís'

# Mostrar el mapa interactivo de la colonia en el notebook
mostrar_mapa_colonia(nombre_colonia)